{'mail': 'tarasov.rb@phystech.edu',

 'id': 324115777,
 
 'type': 'regression',
 
 'dataset': {'name': 'Diabetes Data Set',
 
  'url': 'https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_diabetes.html#sklearn.datasets.load_diabetes'},
  
 'method': 'Надарая-Ватсона', 'SVR', 'Линейная регрессия'}

In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from scipy.spatial.distance import cdist

In [2]:
class FNV(object):
    def __init__(self, kernel=None):
        self.X, self.Y = None, None
        self.kernel = lambda x: np.ones_like(x)
        if kernel is not None:
            self.kernel = kernel
    def predict(self, X):
        features = np.sum(self.Y*self.kernel(cdist(X, self.X)), axis=-1)
        return features/(np.sum(self.kernel(cdist(X, self.X)), axis=-1)+1e-10)
    def fit(self, X, Y, epoch=10):
        self.X, self.Y = np.array(X), np.array(Y)

In [3]:
data = load_diabetes()

In [4]:
data.feature_names

['age', 'sex', 'bmi', 'bp', 's1', 's2', 's3', 's4', 's5', 's6']

Посмотрим информацию о признаках. 

In [5]:
print(data.DESCR)

.. _diabetes_dataset:

Diabetes dataset
----------------

Ten baseline variables, age, sex, body mass index, average blood
pressure, and six blood serum measurements were obtained for each of n =
442 diabetes patients, as well as the response of interest, a
quantitative measure of disease progression one year after baseline.

**Data Set Characteristics:**

  :Number of Instances: 442

  :Number of Attributes: First 10 columns are numeric predictive values

  :Target: Column 11 is a quantitative measure of disease progression one year after baseline

  :Attribute Information:
      - age     age in years
      - sex
      - bmi     body mass index
      - bp      average blood pressure
      - s1      tc, T-Cells (a type of white blood cells)
      - s2      ldl, low-density lipoproteins
      - s3      hdl, high-density lipoproteins
      - s4      tch, thyroid stimulating hormone
      - s5      ltg, lamotrigine
      - s6      glu, blood sugar level

Note: Each of these 10 feature va

Как видим, их всего 10, из них признак 'sex' категориальный, остальные вещественные. Но в наших загруженных данных 'sex' уже преобразован в вещественный. 

Целевой признак вещественный --- степень прогресса болезни. Имеем дело с задачей регрессии.

In [6]:
X = data.data
Y = data.target

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size = 0.8, random_state=42)

In [7]:
print(min(Y), max(Y))

25.0 346.0


Выполним нормировку данных

In [8]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

### Линейная регрессия

Попробуем сначала линейную регрессию. В качестве метрики качества возьмём MSE. Также будем выводить mean absolute percentage error -- на какую долю в среднем мы ошибаемся.

Сначала запустим без регуляризации:

In [9]:
model = LinearRegression()
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2900.173287883232
MAPE = 0.3266573645839748


Теперь с L2 и L1-регуляризацией. Экспериментально установил, что при коэффициентах регуляризации alpha_ridge = 100 и alpha_lasso = 2 у нас наилучшая точность.

In [10]:
model = Ridge(alpha = 100)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2858.2204051303684
MAPE = 0.31158070937800936


In [11]:
model = Lasso(alpha = 2)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))
model.coef_

MSE = 2798.849444458188
MAPE = 0.31275185486547474


array([  0.        ,  -7.42688613,  26.14131077,  14.7357929 ,
        -4.06586241,  -0.        , -10.87127975,   0.        ,
        21.50433126,   1.46725679])

### Метод опорных векторов

Посмотрим на SVR с различными ядрами. Оптимальные коэффициенты регуляризации подобраны экспериментально.

In [12]:
model = SVR(kernel = 'linear')
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2939.834541736351
MAPE = 0.32409345729775185


In [13]:
model = SVR(kernel = 'linear', C=2)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2934.55335133381
MAPE = 0.32057667361987113


In [14]:
model = SVR(kernel = 'poly', degree = 3, C=4)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 3645.2277738992616
MAPE = 0.3513778935190822


In [15]:
model = SVR(kernel = 'rbf',  C=42)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2508.779062049636
MAPE = 0.3030605694634442


In [16]:
model = SVR(kernel = 'sigmoid', C=5)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2905.7765333603047
MAPE = 0.31528215717265623


### Формула Надарая-Ватсона

Теперь попробуем по формуле Надарая-Ватсона с различными ядрами. Оптимальное значение ширины окна $h$ подбираем экспериментально. 

In [17]:
def K_square(distance, h=0.2): # Квадратичное
    ret = np.array(distance)/h
    return (1 - ret**2) * (np.abs(ret) <= 1)

model = FNV(kernel = lambda x: K_square(x, h=3))
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 3075.4099118099402
MAPE = 0.3238051949256383


In [18]:
def K_rect(distance, h=0.2): # Прямоугольное
    ret = np.array(distance)/h
    return (np.abs(ret) <= 1)

model = FNV(kernel = lambda x: K_rect(x, h=3))
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 3158.320815001298
MAPE = 0.32897358098522905


In [19]:
def K_trian(distance, h=0.2): # Треугольное
    ret = np.array(distance)/h
    return (1 - abs(ret)) * (np.abs(ret) <= 1)

model = FNV(kernel = lambda x: K_trian(x, h=3))
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 3059.064461140871
MAPE = 0.32257274355744364


In [20]:
def K_quartic(distance, h=0.2): # Квартическое
    ret = np.array(distance) / h
    return (1 - ret**2)**2 * (np.abs(ret) <= 1)

model = FNV(kernel = lambda x: K_quartic(x, h=3))
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 3039.178193643514
MAPE = 0.3195826583379447


In [21]:
def K_exp(distance): # Гауссовское
    ret = np.array(distance)
    return math.e ** (-2 * ret**2)

model = FNV(kernel = K_exp)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 3278.2907981940225
MAPE = 0.33281302922788747


### Отбор признаков

Сделаем отбор признаков, убирая те из них, для которых зануляются коэффициенты при LASSO-регуляризации. Наименьшее значние у MSE было когда при LASSO-регуляризации занулились признаки 0, 5, 7. Экспериментально установил, что сначала зануляется коэффициент при признаке номер 5, затем при 7 и при 0.

Сначала уберём пятый признак. И протестируем без него некоторые модели.

In [22]:
Z = np.array([(X.T)[0], (X.T)[1], (X.T)[2], (X.T)[3], (X.T)[4], (X.T)[6], (X.T)[7], (X.T)[8], (X.T)[9]]).T # Without 5
Z.shape

(442, 9)

In [23]:
Z_train, Z_test, Y_train, Y_test = train_test_split(Z, Y, train_size = 0.8, random_state=42)

In [24]:
model = LinearRegression()
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2900.243154190635
MAPE = 0.32076193465343467


In [25]:
model = SVR(kernel = 'rbf', C = 35)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2389.5159643133843
MAPE = 0.29098916808050734


In [26]:
model = Ridge(alpha = 0.2)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2847.9679284806266
MAPE = 0.3112884858184706


In [27]:
model = Lasso(alpha = 0.1)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))
model.coef_

MSE = 2798.2394798511846
MAPE = 0.312681066374868


array([   0.        , -152.6359861 ,  552.75020915,  303.37206485,
        -81.37917519, -229.23046556,    0.        ,  447.90663478,
         29.63249077])

Видим, что точность предсказания улучшилась

Теперь уберём ещё и седьмой

In [28]:
Z = np.array([(X.T)[0], (X.T)[1], (X.T)[2], (X.T)[3], (X.T)[4], (X.T)[6],  (X.T)[8], (X.T)[9]]).T # Without 5, 7
Z.shape

(442, 8)

In [29]:
Z_train, Z_test, Y_train, Y_test = train_test_split(Z, Y, train_size = 0.8, random_state=42)

In [30]:
model = LinearRegression()
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2858.8990399182303
MAPE = 0.3205026936921082


In [31]:
model = SVR(kernel = 'rbf', C = 30)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2390.9261830299783
MAPE = 0.28941820902209175


In [32]:
model = Ridge(alpha = 0.15)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2825.5730204436722
MAPE = 0.3112771384975224


In [33]:
model = Lasso(alpha = 0.1)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))
model.coef_

MSE = 2798.2399133966
MAPE = 0.31268110966491425


array([   0.        , -152.63561565,  552.75074426,  303.37174318,
        -81.37970821, -229.22979872,  447.90646746,   29.63277949])

Уберём нулевой

In [34]:
Z = np.array([ (X.T)[1], (X.T)[2], (X.T)[3], (X.T)[4], (X.T)[6],  (X.T)[8], (X.T)[9]]).T # Without 0, 5, 7
Z.shape

(442, 7)

In [35]:
Z_train, Z_test, Y_train, Y_test = train_test_split(Z, Y, train_size = 0.8, random_state=42)

In [36]:
model = LinearRegression()
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2828.897521754372
MAPE = 0.3171342536562802


In [37]:
model = SVR(kernel = 'rbf', C = 25)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2460.3982243679643
MAPE = 0.29751114184855315


In [38]:
model = Ridge(alpha = 0.15)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2796.737409747768
MAPE = 0.30735821937766866


In [39]:
model = Lasso(alpha = 0.1)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))
model.coef_

MSE = 2798.2404515363482
MAPE = 0.3126803083970271


array([-152.63017134,  552.75373077,  303.35883942,  -81.39898141,
       -229.21037658,  447.90764357,   29.64673454])

In [40]:
def K_exp(distance): # Гауссовское ядро, метод Надарая-Ватсона
    ret = np.array(distance)
    return math.e ** (-2 * ret**2)

model = FNV(kernel = K_exp)
model.fit(X_train, Y_train)
Y_pred = model.predict(X_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 3278.2907981940225
MAPE = 0.33281302922788747


### Уменьшение размерности

Теперь применим SVD-разложение для уменьшения размерности.

In [41]:
U, D, Vh = np.linalg.svd(X, full_matrices=False)
np.allclose(X, np.dot(U, np.dot(np.diag(D), Vh)))

True

In [42]:
D

array([2.00604441, 1.22160478, 1.09816315, 0.97748473, 0.81374786,
       0.77634993, 0.73250287, 0.65854628, 0.27985695, 0.09252313])

Последний элемент сильно меньше остальных. Уберём его из матрицы, занулив. Затем Если мы перемножим наши матрицы $U, D$ и $Vh$, получим новую матрицу размером $442 \times 10$ ранга $9$. Уберём из неё один столбец и получим новую матрицу признаков $Z \in \mathbb{R}^{442\times 9}$.

In [43]:
D = D[:9]
Vh = Vh[:9]
U = (U.T[:9]).T
U.shape
Z = np.dot(U, np.dot(np.diag(D), Vh))
Z = (Z.T[:9]).T
Z_train, Z_test, Y_train, Y_test = train_test_split(Z, Y, train_size = 0.8, random_state=42)

Теперь попробуем некоторые алгоритмы, которые дали хорошие результаты на данных без уменьшения размерности.

In [44]:
model = Ridge(alpha = 0.2)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2870.81598415855
MAPE = 0.31234120272441174


In [45]:
model = Lasso(alpha = 0.1)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))
model.coef_

MSE = 2811.099199927725
MAPE = 0.3119259513718658


array([   0.        , -152.80500199,  558.69065576,  310.19571558,
         -0.        ,  -64.75658458, -267.40724215,    0.        ,
        416.96362258])

In [46]:
model = SVR(kernel = 'sigmoid', C=5)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2858.3909260754385
MAPE = 0.3153711110862592


In [47]:
model = SVR(kernel = 'rbf', C = 37)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 2510.5900432461062
MAPE = 0.2956763477630805


In [48]:
def K_quartic(distance, h=0.1): # Квартическое
    ret = np.array(distance) / h
    return (1 - ret**2)**2 * (np.abs(ret) <= 1)

model = FNV(kernel = lambda x: K_quartic(x, h=0.2))
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 3172.8227456284158
MAPE = 0.3282094538334827


Отметим, что размер окна для квартического ядра заметно уменьшился по сравнению с тем, который был до уменьшения размерности.

In [49]:
def K_exp(distance): # Гауссовское
    ret = np.array(distance)
    return math.e ** (-2 * ret**2)

model = FNV(kernel = K_exp)
model.fit(Z_train, Y_train)
Y_pred = model.predict(Z_test)
print("MSE =", mean_squared_error(Y_pred, Y_test))
print("MAPE =", mean_absolute_percentage_error(Y_pred, Y_test))

MSE = 5224.4195832573105
MAPE = 0.4118102811944167


### Summary

Лучший результат и до и после отбора признаков и уменьшения размерности дало rbf ядро в методе опорных векторов. Это подтверждает его универсальность. Также неплохой результат дала линейная регрессия. 

Отбор признаков и уменьшение размерности привели к улучшению результата для линейной регрессии и SVR. Это говорит о том, что есть признаки, которые почти не влияют на результат или коррелируют с другими признаками.

Когда мы убрали лишний признкак, линейная регрессия стала давать лучший результат при меньших коэффициентах регуляризации. Это говорит о том, что признаки меньше стали коррелировать, и веса растут не так быстро.

Наменьшее значение $MSE$, которую получили среди всех этих методов --- $MSE \approx 2389$. Достигнуто на SVR после того как был убран один из признаков. При этом $MAPE \approx 0.3$. Целевой признак в нашей выборке пробегает значения от $25$ до $346$. То есть, даже лучшая из наших моделей предсказыает довольно грубо. Значит, для нормального предсказания надо выбрать более сложные модели.